<a href="https://colab.research.google.com/github/RubenGres/AerialDreams/blob/main/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob

In [2]:
aerial_tif = glob.glob("flair_1_toy_aerial_test/*/*/img/*.tif")
mask_tif = glob.glob("flair_1_toy_labels_test/*/*/msk/*.tif")

In [3]:
from osgeo import gdal
import numpy as np
from PIL import Image

def convert_image(input_path, output_path):
    input_tif = gdal.Open(input_path)
    rgb_data = np.zeros((input_tif.RasterYSize, input_tif.RasterXSize, 3), dtype=np.uint8)

    for i in range(3):
        rgb_data[..., i] = input_tif.GetRasterBand(i+1).ReadAsArray()

    # Close input TIFF file
    input_tif = None

    img = Image.fromarray(rgb_data)
    img.save(output_path)

In [4]:
import json
f = open('/content/flair-1_metadata_aerial.json', 'r')
metadata = json.load(f)

In [7]:
meta = metadata["IMG_063104"]
!python AerialDreams/scripts/get_metadata.py /content/flair_1_toy_lables_test/D012_2019/Z13_AA/msk/IMG_062871.tif tkt.json "{meta}"

{'place_id': 124129123, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 80715917, 'lat': '44.05644378518927', 'lon': '3.2112661215780682', 'display_name': 'Millau, Aveyron, Occitanie, France métropolitaine, 12100, France', 'address': {'town': 'Millau', 'municipality': 'Millau', 'county': 'Aveyron', 'ISO3166-2-lvl6': 'FR-12', 'state': 'Occitanie', 'ISO3166-2-lvl4': 'FR-OCC', 'region': 'France métropolitaine', 'postcode': '12100', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['44.0500761', '44.0572029', '3.2002776', '3.2289643']}


In [8]:
import os
from tqdm import tqdm

dataset_path = 'dataset'

for img, msk in tqdm(zip(aerial_tif, mask_tif)):
  basename = os.path.basename(img)
  filename_img = os.path.basename(img).split('.')[0]
  filename_msk = os.path.basename(msk).split('.')[0]
  number = filename_img.split('_')[1]

  folder = number[0:3]

  base_dir = f"{dataset_path}/{folder}/{number}"

  os.makedirs(base_dir, exist_ok = True)

  mask_out = f"{base_dir}/{filename_msk}.png"
  image_out = f"{base_dir}/{filename_img}.png"
  json_out = f"{base_dir}/{number}.json"
  percentages_out = f"{base_dir}/PCT_{number}.json"

  meta = metadata[filename_img]

  convert_image(img, image_out)
  !python AerialDreams/scripts/get_image_segmentation.py {msk} {mask_out}
  !python AerialDreams/scripts/get_metadata.py {img} {json_out} "{meta}"
  !python AerialDreams/scripts/get_classes_percentages.py {mask_out} {percentages_out}

{'place_id': 129975463, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 108791591, 'lat': '44.9041083', 'lon': '4.8807118', 'display_name': 'Valence, Drôme, Auvergne-Rhône-Alpes, France métropolitaine, 26000, France', 'address': {'city': 'Valence', 'municipality': 'Valence', 'county': 'Drôme', 'ISO3166-2-lvl6': 'FR-26', 'state': 'Auvergne-Rhône-Alpes', 'ISO3166-2-lvl4': 'FR-ARA', 'region': 'France métropolitaine', 'postcode': '26000', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['44.9036907', '44.9041358', '4.8785763', '4.8807118']}


45it [02:53,  3.92s/it]

{'place_id': 115169603, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 37796013, 'lat': '43.47210058152163', 'lon': '-1.5656145474840815', 'display_name': 'Rue Georges Hennebutte, Lapurdi, Biarritz, Bayonne, Pyrénées-Atlantiques, Nouvelle-Aquitaine, France métropolitaine, 64200, France', 'address': {'road': 'Rue Georges Hennebutte', 'subdivision': 'Lapurdi', 'town': 'Biarritz', 'municipality': 'Bayonne', 'county': 'Pyrénées-Atlantiques', 'ISO3166-2-lvl6': 'FR-64', 'state': 'Nouvelle-Aquitaine', 'ISO3166-2-lvl4': 'FR-NAQ', 'region': 'France métropolitaine', 'postcode': '64200', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['43.4720084', '43.4729632', '-1.5665961', '-1.5643931']}


46it [02:57,  3.95s/it]

{'place_id': 338839110, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 9918886373, 'lat': '43.5236884', 'lon': '-1.5037859', 'display_name': "Maison Bicicletta, Avenue de l'Adour, Lapurdi, Brise Lames, Anglet, Bayonne, Pyrénées-Atlantiques, Nouvelle-Aquitaine, France métropolitaine, 64600, France", 'address': {'shop': 'Maison Bicicletta', 'road': "Avenue de l'Adour", 'subdivision': 'Lapurdi', 'suburb': 'Brise Lames', 'town': 'Anglet', 'municipality': 'Bayonne', 'county': 'Pyrénées-Atlantiques', 'ISO3166-2-lvl6': 'FR-64', 'state': 'Nouvelle-Aquitaine', 'ISO3166-2-lvl4': 'FR-NAQ', 'region': 'France métropolitaine', 'postcode': '64600', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['43.5236384', '43.5237384', '-1.5038359', '-1.5037359']}


...

48it [03:04,  3.71s/it]

{'place_id': 123688214, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 75837685, 'lat': '42.808883159717375', 'lon': '-0.5581442411076705', 'display_name': "Piste de la Cabane d'Anglus, Borce, Oloron-Sainte-Marie, Pyrénées-Atlantiques, Nouvelle-Aquitaine, France métropolitaine, 64490, France", 'address': {'road': "Piste de la Cabane d'Anglus", 'village': 'Borce', 'municipality': 'Oloron-Sainte-Marie', 'county': 'Pyrénées-Atlantiques', 'ISO3166-2-lvl6': 'FR-64', 'state': 'Nouvelle-Aquitaine', 'ISO3166-2-lvl4': 'FR-NAQ', 'region': 'France métropolitaine', 'postcode': '64490', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['42.8080474', '42.8112745', '-0.5620035', '-0.557372']}


49it [03:09,  4.03s/it]

{'place_id': 85078176, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 8483571797, 'lat': '43.2867479', 'lon': '-0.3564063', 'display_name': "29, Avenue de l'Yser, Bizanos, Pau, Pyrénées-Atlantiques, Nouvelle-Aquitaine, France métropolitaine, 64320, France", 'address': {'house_number': '29', 'road': "Avenue de l'Yser", 'village': 'Bizanos', 'municipality': 'Pau', 'county': 'Pyrénées-Atlantiques', 'ISO3166-2-lvl6': 'FR-64', 'state': 'Nouvelle-Aquitaine', 'ISO3166-2-lvl4': 'FR-NAQ', 'region': 'France métropolitaine', 'postcode': '64320', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['43.2866979', '43.2867979', '-0.3564563', '-0.3563563']}


50it [03:12,  3.86s/it]
